For this assignment, use the dataset called `radar_parameters.csv` provided in the GitHub repository in the folder `homework`.
The provided data has 
## Dataset Description

The training data consists of polarimetric radar parameters calculated from a disdrometer (an instrument that measures rain drop sizes, shapes, and rainfall rate) measurements from several years in Huntsville, Alabama. A model called `pytmatrix` is used to calculate polarimetric radar parameters from the droplet observations, which can be used as a way to compare what a remote sensing instrument would see and rainfall.

## Data columns

Features (radar measurements):

`Zh` - radar reflectivity factor (dBZ) - use the formula $dBZ = 10\log_{10}(Z)$

`Zdr` - differential reflectivity

`Ldr` - linear depolarization ratio

`Kdp` - specific differential phase

`Ah` - specific attenuation

`Adp` - differential attenuation

Target :

`R` - rain rate


1. Split the data into a 70-30 split for training and testing data.

In [2]:
# import needed packages
import seaborn as sns
import pandas as pd
import numpy as np 
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

In [3]:
# open up csv
radar_ds = pd.read_csv('radar_parameters.csv')
radar_ds

,Unnamed: 0,Zh (dBZ),Zdr (dB),Ldr (dB),Kdp (deg km-1),Ah (dBZ/km),Adr (dB/km),R (mm/hr)
0,0,23.144878,0.418637,-41.757733,0.005395,0.000290,0.000012,2.393520
1,1,22.737156,0.322850,-43.772069,0.005194,0.000360,0.000012,3.502699
2,2,26.869826,0.330948,-43.577399,0.013385,0.000903,0.000030,8.627561
3,3,28.540561,0.399480,-42.139731,0.018872,0.001036,0.000043,8.424447
4,4,30.500127,0.543758,-39.763087,0.027438,0.001157,0.000064,8.189291
...,...,...,...,...,...,...,...,...
18964,18964,31.515997,0.579955,-39.244229,0.034048,0.001417,0.000080,10.648020
18965,18965,29.993334,0.567935,-39.399188,0.024134,0.001032,0.000057,7.981875
18966,18966,31.685913,0.655681,-38.375696,0.033971,0.001165,0.000081,6.822691
18967,18967,32.980096,0.768586,-37.166218,0.043117,0.001285,0.000105,6.801169


In [4]:
# retrieve x and y data 
x_radar = radar_ds[['Zh (dBZ)', 'Zdr (dB)', 'Ldr (dB)', 'Kdp (deg km-1)', 'Ah (dBZ/km)', 'Adr (dB/km)']]  # Features
y_radar = radar_ds['R (mm/hr)']
y_radar

0         2.393520
1         3.502699
2         8.627561
3         8.424447
4         8.189291
           ...    
18964    10.648020
18965     7.981875
18966     6.822691
18967     6.801169
18968     2.582440
Name: R (mm/hr), Length: 18969, dtype: float64

In [5]:
# split 70/30 train-test
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(x_radar, y_radar,
                                                random_state=1, test_size=0.3)


2. Using the split created in (1), train a multiple linear regression dataset using the training dataset, and validate it using the testing dataset.  Compare the $R^2$ and root mean square errors of model on the training and testing sets to a baseline prediction of rain rate using the formula $Z = 200 R^{1.6}$.

In [6]:
# create model for linear regression
model = LinearRegression(fit_intercept=True)
model.fit(xtrain, ytrain)

# evaluate the model on the second set of data
ypred_test = model.predict(xtest)
ypred_train = model.predict(xtrain)

# calculate R^2 and RMSE for the model on the training
r2_train = r2_score(ytrain, ypred_train)
rmse_train = np.sqrt(mean_squared_error(ytrain, ypred_train))

# calculate R^2 and RMSE for the model on the testing
r2_test = r2_score(ytest, ypred_test)
rmse_test = np.sqrt(mean_squared_error(ytest, ypred_test))

# calculate baseline prediction using the formula
baseline_pred_train = 200 * (ytrain ** 1.6)
baseline_pred_test = 200 * (ytest ** 1.6)

# calculate R^2 and RMSE for the baseline prediction on the training
baseline_r2_train = r2_score(ytrain, baseline_pred_train)
baseline_rmse_train = np.sqrt(mean_squared_error(ytrain, baseline_pred_train))

# calculate R^2 and RMSE for the baseline prediction on the testing
baseline_r2_test = r2_score(ytest, baseline_pred_test)
baseline_rmse_test = np.sqrt(mean_squared_error(ytest, baseline_pred_test))

print("r2_train:", r2_train)
print("rmse_train:", rmse_train)
print("baseline_r2_train:", baseline_r2_train)
print("baseline_rmse_train:", baseline_rmse_train)

print("r2_test:", r2_test)
print("rmse_test", rmse_test)
print("baseline_r2_test:", baseline_r2_test)
print("baseline_rmse_test:", baseline_rmse_test)


r2_train: 0.9876962938881576
rmse_train: 0.9411927035424521
baseline_r2_train: -6962117.746901507
baseline_rmse_train: 22388.824417465512
r2_test: 0.9896313936002781
rmse_test 0.8922106275440649
baseline_r2_test: -7039158.818815004
baseline_rmse_test: 23247.03719754933


3. Repeat 1 doing a grid search over polynomial orders, using a grid search over orders 0-21, and use cross-validation of 7 folds.  For the best polynomial model in terms of $R^2$, does it outperform the baseline and the linear regression model in terms of $R^2$ and root mean square error?

In [7]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures

In [8]:
# Define the parameter grid for the grid search
param_grid = {
    'polynomialfeatures__degree': np.arange(0, 8)
    }

# Create a pipeline with PolynomialFeatures and LinearRegression
pipeline = make_pipeline(PolynomialFeatures(), LinearRegression())

# Perform grid search
grid_search_poly = GridSearchCV(pipeline, param_grid, cv=7, scoring='r2')
grid_search_poly.fit(xtrain, ytrain)

# Evaluate the best model on the testing data
best_model = grid_search_poly.best_estimator_
best_pred_train_poly = best_model.predict(xtrain)
best_pred_test_poly = best_model.predict(xtest)
best_rmse_train_poly = np.sqrt(mean_squared_error(ytrain, best_pred_train_poly))
best_rmse_test_poly = np.sqrt(mean_squared_error(ytest, best_pred_test_poly))
best_r2_train_poly = r2_score(ytrain, best_pred_train_poly)
best_r2_test_poly = r2_score(ytest, best_pred_test_poly)
test_score = best_model.score(xtest, ytest)

print("Best poly regression rmse (train):", best_rmse_train_poly)
print("Best poly regression rmse (test):", best_rmse_test_poly)
print("Best poly regression r2 (train):", best_r2_train_poly)
print("Best poly regression r2 (test):", best_r2_test_poly)

print("Baseline_r2_test:", baseline_r2_test)
print("Baseline_rmse_test:", baseline_rmse_test)



Best poly regression rmse (train): 0.009948165588759274
Best poly regression rmse (test): 0.03690123051696115
Best poly regression r2 (train): 0.9999986254363445
Best poly regression r2 (test): 0.9999822635253892
Baseline_r2_test: -7039158.818815004
Baseline_rmse_test: 23247.03719754933


4. Repeat 1 with a Random Forest Regressor, and perform a grid_search on the following parameters:
   
   ```python
   {'bootstrap': [True, False],  
   'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],  
   'max_features': ['auto', 'sqrt'],  
   'min_samples_leaf': [1, 2, 4],  
   'min_samples_split': [2, 5, 10],  
   'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}
   ```
  Can you beat the baseline, or the linear regression, or best polynomial model with the best optimized Random Forest Regressor in terms of $R^2$ and root mean square error?

In [8]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor

In [10]:
param_grid_2 = {
   'bootstrap': [True, False],  
   'max_depth': [10, 20, 30], 
                 #40, 50, 60, 70, 80, 90, 100, None],  
   'max_features': ['auto', 'sqrt'],  
   'min_samples_leaf': [1, 2, 4],  
   'min_samples_split': [2, 5, 10],  
   'n_estimators': [200, 400, 600]
                    #800, 1000, 1200, 1400, 1600, 1800, 2000]
   }

In [11]:
# BUild the model! Create and train the random forest classifier
rf = RandomForestRegressor()

# Perform grid search
grid_search_rf = GridSearchCV(rf, param_grid_2, cv=7, scoring='r2')
grid_search_rf.fit(xtrain, ytrain)

best_model_rf = grid_search_rf.best_estimator_
best_pred_test_rf = best_model_rf.predict(xtest)
best_pred_train_rf = best_model_rf.predict(xtrain)
best_rmse_train_rf = np.sqrt(mean_squared_error(ytrain, best_pred_train_rf))
best_rmse_test_rf = np.sqrt(mean_squared_error(ytest, best_pred_test_rf))
best_r2_train_rf = r2_score(ytrain, best_pred_train_rf)
best_r2_test_rf = r2_score(ytest, best_pred_test_rf)
test_score_rf = best_model_rf.score(xtest, ytest)

print("Best Random Forest Regressor RMSE Train:", best_rmse_train_rf)
print("Best Random Forest Regressor RMSE Test:", best_rmse_test_rf)
print("Best Random Forest Regressor R2 Train:", best_r2_train_rf)
print("Best Random Forest Regressor R2 Train:", best_r2_test_rf)


